In [1]:
!pip install wandb
!pip install --upgrade pandas

  Using cached wandb-0.10.13-py2.py3-none-any.whl (1.9 MB)
Processing /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a/subprocess32-3.5.4-py3-none-any.whl
Processing /root/.cache/pip/wheels/29/93/c6/762e359f8cb6a5b69c72235d798804cae523bbe41c2aa8333d/promise-2.3-py3-none-any.whl
  Using cached shortuuid-1.0.1-py3-none-any.whl (7.5 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached GitPython-3.1.12-py3-none-any.whl (159 kB)
  Using cached sentry_sdk-0.19.5-py2.py3-none-any.whl (128 kB)
  Using cached configparser-5.0.1-py3-none-any.whl (22 kB)
  Using cached gitdb-4.0.5-py3-none-any.whl (63 kB)
  Using cached smmap-3.0.4-py2.py3-none-any.whl (25 kB)


In [1]:
import numpy as np
import pandas as pd
import sagemaker
from sagemaker.pytorch import PyTorch
from sklearn.metrics import classification_report, confusion_matrix, f1_score, matthews_corrcoef
import wandb

In [17]:
session = sagemaker.Session()
role = sagemaker.get_execution_role()
BUCKET = 'sagemaker-capra'
BASE_PATH = f's3://{BUCKET}'
SAVE_PATH = f'{BASE_PATH}/models/norec/simple_transformers'
DATA_PATH = f'{BASE_PATH}/data/norec'

In [19]:
subset_names = ['train', 'test', 'dev']
subsets = {name: pd.read_pickle(f'{DATA_PATH}/norsk_kategori_4_{name}.pkl') for name in subset_names}

In [20]:
subsets['train'].groupby(['rating']).count()

,text
rating,
0,2326
1,11597


In [58]:
subsets['train'].head(1)

,text,label
2676,«Poison». Som alle store artister passer Timbe...,1


In [22]:
subsets = {name: subsets[name].rename(columns={'rating': 'label'}) for name in subset_names}
subsets['train'].head(1)

,text,label
2676,«Poison». Som alle store artister passer Timbe...,1


In [77]:
negative_samples = subsets['train'][subsets['train']['label'] == 0]
negative_length = len(negative_samples)
negative_length

2326

In [78]:
positive_samples = subsets['train'][subsets['train']['label'] == 1]
positive_length = len(positive_samples)
positive_length

11597

In [80]:
shortest_lenght = min(negative_length, positive_length)
shortest_lenght

2326

In [82]:
negative_samples = negative_samples.sample(n = shortest_lenght)
len(negative_samples)

2326

In [83]:
positive_samples = positive_samples.sample(n = shortest_lenght)
len(positive_samples)

2326

In [85]:
balanced_df = pd.concat([negative_samples, positive_samples]).sample(frac=1.0)
balanced_df.head(5)

,text,label
1373,The Love Guru. Gudene skal vite at Mike Myers ...,0
8504,- Mitt favoritt-album fra Beyoncé. ALBUM:. Som...,1
12170,Harry var en opprører. I 1971 var lufta over S...,1
3625,"The High Llamas:. «Beat, Maize & Corn». Symfon...",1
1575,Scoreturen. Har du noen sinne hørt en mer foru...,0


In [94]:
negative_length / positive_length

0.20056911270156075

In [22]:
print(f'Training set consists of {len(subsets["train"])} samples')
print(f'Dev set consists of {len(subsets["dev"])} samples')
print(f'Test set consists of {len(subsets["test"])} samples')

Training set consists of 17502 samples
Dev set consists of 2239 samples


In [23]:
wandb.sagemaker_auth(path='source')

In [126]:
NUM_EPOCHS = 10
LEARNING_RATE = 0.00001

In [127]:
training_args = {
    'balance-training-set': True,
    'batch-size': 8,
    'evaluate-during-training': False,
    'label-weights': '1 1',
    'num-epochs': NUM_EPOCHS,
    'learning-rate': LEARNING_RATE,
    'overwrite-output-dir': True,
    'save-model-every-epoch': False,
}

In [128]:
pytorch_estimator = PyTorch('simple_transformers_train.py',
                            source_dir='source',
                            instance_type='ml.p2.xlarge',
                            instance_count=1,
                            framework_version='1.6.0',
                            py_version='py3',
                            role=role,
                            hyperparameters = training_args)
pytorch_estimator.fit({'training': DATA_PATH})

2021-01-20 14:28:44 Starting - Starting the training job...
2021-01-20 14:29:07 Starting - Launching requested ML instancesProfilerReport-1611152924: InProgress
......
2021-01-20 14:30:07 Starting - Preparing the instances for training......
2021-01-20 14:31:15 Downloading - Downloading input data...
2021-01-20 14:31:31 Training - Downloading the training image............
2021-01-20 14:33:44 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-01-20 14:33:44,419 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-01-20 14:33:44,462 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-01-20 14:33:47,497 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-01-20 14:33:47,911 sagemaker-training-toolkit INFO     Installing dependencies from requir

UnexpectedStatusException: Error for Training job pytorch-training-2021-01-20-14-28-43-818: Failed. Reason: ClientError: Artifact upload failed:Insufficient disk space